# Streetaware - Object Detection Data

In [8]:
import pandas as pd
import numpy as np
import time
import json
import os

np.random.seed(2024)

from visualizations import plot_topomap_comparison_highlight, plot_hierarchical_treemap

from TopoTree import TopoTree
from TopoMap import TopoMap
from HierarchicalTopoMap import HierarchicalTopoMap

In [3]:
def process_embedding_data(base_path, street, side):
    embeddings_path = base_path+f'{street}_1_sensor_1_{side}_crop_embeddings.pkl'
    data_path = base_path+f'{side}_detections.json'

    with open(embeddings_path, 'rb') as f:
        original_emb = np.load(f, allow_pickle=True)

    with open(data_path, 'rb') as f:
        data = json.load(f)
        
    list_emb = []
    list_classes = []

    n_frames = len(original_emb)

    for frame in range(n_frames):
        emb_frame = original_emb[frame]
        n_obj_frame = len(emb_frame)

        for i in range(n_obj_frame):
            list_emb.append(emb_frame[i,:])

        list_classes.extend(data[frame]['classIDs'])

    all_emb = np.array(list_emb)
    all_classes = np.array(list_classes)

    select_classes = [0, #peson
                      1, #bicycle
                      2, #car
                     ]

    emb = np.zeros((0, 768))
    classes = []

    for c in select_classes:
        mask_class = all_classes == c
        emb = np.vstack([emb, all_emb[mask_class,:]])
        classes.extend(mask_class.sum()*[c])

    classes = np.array(classes)

    return emb, classes

In [9]:
street = 'chase'
side = 'left'
base_path = f'data/streetaware/{street}/'

if os.path.isfile(base_path+'embeddings.npy'):
    with open(base_path+'embeddings.npy', 'rb') as f:
        emb = np.load(f)
    with open(base_path+'classes.npy', 'rb') as f:
        classes = np.load(f)

else:
    emb, classes = process_embedding_data(base_path, street, side)

    with open(base_path+'embeddings.npy', 'wb') as f:
        np.save(f, emb)

    with open(base_path+'classes.npy', 'wb') as f:
        np.save(f, classes)


In [10]:
n_points = emb.shape[0]

print(f'Total number of objects: {n_points}')

samples = np.random.randint(0, n_points, 10000)

emb_sample = emb[samples,:]
classes_sample = classes[samples]

Total number of objects: 363134


In [11]:
start_time = time.time()

topomap_samples = TopoMap(emb_sample)
proj_topomap_samples = topomap_samples.run()

topomap_time = time.time()-start_time
print(f'Time for runinning TopoMap: {topomap_time}s')

Time for runinning TopoMap: 230.8022701740265s


In [12]:
start_time = time.time()

topotree_samples = TopoTree(emb_sample, min_box_size=0.01*emb_sample.shape[0])
topotree_samples.mst = topomap_samples.mst
topotree_samples.sorted_edges = topomap_samples.sorted_edges
comp_info_samples = topotree_samples.run()

topotree_time = time.time()-start_time
print(f'Time for runinning TopoTree: {topotree_time}s')

df_comp_samples = pd.DataFrame.from_dict(comp_info_samples)

df_comp_samples.head()

Time for runinning TopoTree: 4.326971054077148s


,id,points,size,persistence,created_at,children,parent,died_at,persistence_density
0,0,"[1, 3844, 4361, 3849, 268, 7182, 7696, 5138, 5...",819,2.459050,2.534557,336,7.0,4.993607,164.009717
1,1,"[6401, 7, 8200, 8202, 7180, 9231, 9487, 2579, ...",196,0.282517,2.902093,53,3.0,3.184610,61.546004
2,2,"[2310, 2825, 1035, 2572, 8459, 6160, 3602, 19,...",138,0.003234,3.181376,1,3.0,3.184610,43.333411
3,3,"[7, 5128, 9, 9225, 8202, 7180, 8200, 1035, 923...",701,1.681208,3.184610,192,6.0,4.865818,144.066219
4,4,"[8965, 9222, 4615, 5392, 4370, 3604, 3094, 744...",196,1.448691,3.417127,65,6.0,4.865818,40.280997


In [13]:
fig = plot_hierarchical_treemap(df_comp_samples, color='died_at')
fig.update_layout(title='TopoTree - Streetaware Sample Dataset')
fig.show()

In [14]:
components_to_highligth = [0,6,5,9]

start_time = time.time()

hiertopomap_samples = HierarchicalTopoMap(emb_sample, 
                                          components_to_scale=components_to_highligth)
hiertopomap_samples.min_points_component = topotree_samples.min_box_size
hiertopomap_samples.mst = topotree_samples.mst
hiertopomap_samples.sorted_edges = topotree_samples.sorted_edges
proj_hier_samples = hiertopomap_samples.run()

hiertopomap_time = time.time()-start_time
print(f'Time for runinning HierarchicalTopoMap: {hiertopomap_time}s')

Scalling component 0 - Scale: 9.21094249770779 scaling - initial area: 129854.508... final area: 11017044.000...
Scalling component 6 - Scale: 9.371754406530615 scaling - initial area: 170000.000... final area: 14931064.000...
Scalling component 5 - Scale: 4.800286174906426 scaling - initial area: 3849452.000... final area: 88701952.000...
Scalling component 9 - Scale: 7.14803595941859 scaling - initial area: 15731.492... final area: 803791.438...
[INFO] Number of edges hit. Edges processed: 9998
Time for runinning HierarchicalTopoMap: 37.06816339492798s


In [15]:
fig = plot_topomap_comparison_highlight(proj_topomap_samples, proj_hier_samples, 
                                               components_to_highligth, df_comp_samples,
                                               hiertopomap_samples)
fig.update_layout(height=600, width=1200, title='Streetaware Sample Dataset')
fig.show()